In [1]:
import os
from typing import TypedDict, Literal, Sequence
import asyncio
from decimal import Decimal, ROUND_HALF_DOWN
from dataclasses import dataclass, field, asdict
import json
from pydantic import BaseModel, RootModel
import websockets
from deribit import DERIBIT_TESTNET, DERIBIT_MAINNET
from deribit.auth import AuthedClient, PrivateClient
from deribit.util import round2tick
from dotenv import load_dotenv
load_dotenv()

CLIENT_ID = os.environ['DERIBIT_CLIENT_ID']
CLIENT_SECRET = os.environ['DERIBIT_CLIENT_SECRET']

In [2]:
@dataclass
class Trading:
  client: PrivateClient

  @classmethod
  def new(cls, client_id: str, client_secret: str, *, url: str | None = None, timeout_secs: int | None = None):
    client = PrivateClient.new(client_id, client_secret, url=url, timeout_secs=timeout_secs)
    return cls(client)

  async def __aenter__(self):
    client = await self.client.__aenter__()
    return AuthedTrading(client)
  
  async def __aexit__(self, *args):
    await self.client.__aexit__(*args)

@dataclass
class AuthedTrading:
  client: AuthedClient

  async def buy(
    self, instrument: str, **kwargs    
  ):
    r = await self.client.authed_request({
      'method': 'private/buy',
      'params': {
        'instrument_name': instrument,
        **kwargs
      }
    })
    return r

client = await Trading.new(CLIENT_ID, CLIENT_SECRET, url=DERIBIT_TESTNET).__aenter__()

In [5]:
await client.buy('ETH_USDC', amount=1, price=2000)

{'order': {'label': '',
  'price': 2000.0,
  'direction': 'buy',
  'time_in_force': 'good_til_cancelled',
  'max_show': 1.0,
  'instrument_name': 'ETH_USDC',
  'api': True,
  'web': False,
  'amount': 1.0,
  'order_id': 'ETH_USDC-110886650',
  'creation_timestamp': 1741981590334,
  'mmp': False,
  'replaced': False,
  'filled_amount': 1.0,
  'last_update_timestamp': 1741981590334,
  'post_only': False,
  'average_price': 1357.9,
  'contracts': 10000.0,
  'order_state': 'filled',
  'order_type': 'limit',
  'is_rebalance': False,
  'risk_reducing': False},
 'trades': [{'timestamp': 1741981590334,
   'state': 'filled',
   'price': 1357.9,
   'direction': 'buy',
   'index_price': 1931.632,
   'profit_loss': None,
   'instrument_name': 'ETH_USDC',
   'trade_seq': 18103,
   'api': True,
   'amount': 1.0,
   'mark_price': 1931.632,
   'order_id': 'ETH_USDC-110886650',
   'matching_id': None,
   'tick_direction': 1,
   'fee': 0.0,
   'mmp': False,
   'self_trade': False,
   'post_only': False,

In [8]:
class OrderResult(BaseModel):
    order_id: str
    instrument_name: str
    direction: Literal["buy", "sell"]
    price: Optional[float]
    amount: float
    filled_amount: float
    status: str

class BalanceResult(BaseModel):
    currency: str
    balance: float
    available_funds: float
    margin_balance: float

class SubscriptionResult(BaseModel):
    channel: str
    data: Dict[str, Any]

class OrderParams(TypedDict):
    instrument_name: str
    amount: float
    type: Literal["market", "limit", "stop_limit", "stop_market"]
    price: Optional[float]
    time_in_force: Optional[Literal["good_til_cancelled", "immediate_or_cancel", "fill_or_kill"]]
    post_only: Optional[bool]

class SubscriptionParams(TypedDict):
    channels: Sequence[str]

@dataclass
class Deribit(ClientMixin):
    client_id: Optional[str] = None
    client_secret: Optional[str] = None
    access_token: Optional[str] = None
    testnet: bool = True
    url: str = field(default=DERIBIT_MAINNET, kw_only=True)

    async def __aenter__(self):
        await super().__aenter__()
        if self.client_id and self.client_secret:
            await self.authenticate()
        return self

    async def authenticate(self):
        response = await self.request({
            "method": "public/auth",
            "params": {
                "grant_type": "client_credentials",
                "client_id": self.client_id,
                "client_secret": self.client_secret
            }
        })
        self.access_token = response["access_token"]

    async def private_request(self, method: str, params: dict) -> dict:
        if not self.access_token:
            raise ValueError("Client is not authenticated.")
        params["access_token"] = self.access_token
        return await self.request({"method": f"private/{method}", "params": params})

    async def place_order(self, params: OrderParams) -> OrderResult:
        response = await self.private_request("buy" if params["type"] == "market" else "sell", params)
        return OrderResult(**response)

    async def get_order_state(self, order_id: str) -> OrderResult:
        response = await self.private_request("get_order_state", {"order_id": order_id})
        return OrderResult(**response)

    async def cancel_order(self, order_id: str) -> OrderResult:
        response = await self.private_request("cancel", {"order_id": order_id})
        return OrderResult(**response)

    async def get_balance(self, currency: str) -> BalanceResult:
        response = await self.private_request("get_account_summary", {"currency": currency})
        return BalanceResult(**response)

    async def subscribe(self, params: SubscriptionParams) -> SubscriptionResult:
        response = await self.request({"method": "public/subscribe", "params": params})
        return SubscriptionResult(**response)

    async def unsubscribe(self, params: SubscriptionParams) -> SubscriptionResult:
        response = await self.request({"method": "public/unsubscribe", "params": params})
        return SubscriptionResult(**response)

    @classmethod
    def auth(cls, client_id: str, client_secret: str, testnet: bool = True):
        return cls(client_id=client_id, client_secret=client_secret, testnet=testnet, url=DERIBIT_TESTNET if testnet else DERIBIT_MAINNET)


In [9]:
async with Deribit(CLIENT_ID, CLIENT_SECRET, url=DERIBIT_TESTNET) as client:
  print(await client.get_balance('ETH'))

currency='ETH' balance=1000.0 available_funds=1000.0 margin_balance=1000.0


In [ ]:
@dataclass
class DeribitError(Exception):
  code: int
  message: str
  data: dict | None = None

@dataclass
class Deribit(ClientMixin):
  access_token: str | None = None

  async def auth(self, client_id: str | None = None, client_secret: str | None = None):
    client_id = client_id or os.getenv('DERIBIT_CLIENT_ID')
    client_secret = client_secret or os.getenv('DERIBIT_CLIENT_SECRET')
    if client_id is None or client_secret is None:
      raise ValueError('client_id and client_secret must be provided')
    res = await self.request({
      'method': 'public/auth',
      'params': {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
      }
    })
    if 'error' in res:
      raise DeribitError(**res['error'])
    self.access_token = res['result']['access_token']
    
  async def request(self, msg: dict) -> dict:
    if self.access_token:
      msg = {'access_token': self.access_token, **msg}
    return await super().request(msg)
  
  async def otoco(self, instrument: str, amount: str, buy_price: str, sell_price: str):
    return await self.request({
    'method': 'private/buy',
    'params': {
      'instrument_name': instrument,
      'amount': amount,
      'type': 'limit',
      'price': buy_price,
      'otoco_config': [
        {
          'amount': amount,
          'type': 'limit',
          'price': sell_price,
        }
      ]
    }
  })

In [7]:
client = await Deribit(url=DERIBIT_TESTNET).__aenter__()

In [8]:
await client.auth()

In [ ]:
r = lambda: client.request({
  "method": "public/get_index_price",
  "params": {
      "index_name": "ada_usd"}
})
await asyncio.gather(r(), r(), r())

[{'jsonrpc': '2.0',
  'id': 32,
  'result': {'estimated_delivery_price': 0.7443, 'index_price': 0.7443},
  'usIn': 1741971912578760,
  'usOut': 1741971912578939,
  'usDiff': 179,
  'testnet': True},
 {'jsonrpc': '2.0',
  'id': 33,
  'result': {'estimated_delivery_price': 0.7443, 'index_price': 0.7443},
  'usIn': 1741971912579712,
  'usOut': 1741971912579813,
  'usDiff': 101,
  'testnet': True},
 {'jsonrpc': '2.0',
  'id': 34,
  'result': {'estimated_delivery_price': 0.7443, 'index_price': 0.7443},
  'usIn': 1741971912580753,
  'usOut': 1741971912580925,
  'usDiff': 172,
  'testnet': True}]

In [5]:
total = 450
n_orders = total // 25
print(n_orders)
p_min = Decimal(1895)
p_max = Decimal(1918)

async with Deribit() as client:
  await client.auth()
  for i in range(n_orders):
    buy_price = round2tick((p_max-p_min)*i/n_orders + p_min, Decimal('0.5'))
    sell_price = buy_price + Decimal(0.5)
    print(f'OTOCO. Buy: {buy_price}, Sell: {sell_price}')
    r = await client.otoco('ETH_USDC', '0.009', f'{buy_price:f}', f'{sell_price:f}')

18


DeribitError: 